
# Square Exponential Decomposition Demo

This notebook demonstrates a simple squared exponential (radial basis) covariance decomposition on the square domain [-5, 5] x [-5, 5]. It builds a covariance matrix on a Cartesian grid, performs an eigen-decomposition, exports the truncated representation to JSON, and defines a lightweight loader class that can evaluate or sample the resulting Gaussian field with minimal dependencies.


In [ ]:

import json
from pathlib import Path
import numpy as np



## Configure squared exponential field parameters
The parameters below control the mean, variance, and correlation length of the squared exponential covariance. You can adjust them and rerun the notebook before exporting to JSON.


In [ ]:

# User-adjustable parameters
correlation_length = 1.5
variance = 2.0
mean_value = 0.25

# Domain and grid resolution
extent = (-5.0, 5.0)
grid_points = 31  # per spatial axis; adjust for higher or lower resolution



## Build grid and covariance matrix
We discretize the square using a uniform grid and assemble the squared exponential covariance matrix:

\[ k(x, x') = \sigma^2 \exp\left( - rac{\lVert x - x' Vert^2}{2\ell^2} ight). \]


In [ ]:

x = np.linspace(extent[0], extent[1], grid_points)
y = np.linspace(extent[0], extent[1], grid_points)
xx, yy = np.meshgrid(x, y, indexing="ij")
points = np.column_stack([xx.ravel(), yy.ravel()])

# Pairwise squared distances
p_diff = points[:, None, :] - points[None, :, :]
dist_sq = np.sum(p_diff ** 2, axis=-1)

# Squared exponential covariance matrix
cov = variance * np.exp(-0.5 * dist_sq / (correlation_length ** 2))

cov.shape



## Eigen-decomposition and truncation
We diagonalize the covariance to obtain orthonormal modes. The spectrum is sorted in descending order, and we keep enough modes to capture at least 99% of the total variance (or a minimum of 30 modes to maintain a richer basis at coarse resolutions).


In [ ]:

# Full eigen-decomposition (matrix is symmetric positive-definite)
evals, evecs = np.linalg.eigh(cov)

# Sort descending
idx = np.argsort(evals)[::-1]
evals = evals[idx]
evecs = evecs[:, idx]

# Determine truncation to reach 99% cumulative energy or at least 30 modes
energy = np.cumsum(evals) / np.sum(evals)
min_modes = 30
cutoff_idx = np.searchsorted(energy, 0.99)
keep = max(min_modes, cutoff_idx + 1)

kept_evals = evals[:keep]
kept_evecs = evecs[:, :keep]

keep



## Export decomposition to JSON
The JSON file stores the grid, parameters, eigenvalues, and eigenvectors. This minimal format keeps dependencies to `json` and `numpy` for loading and sampling.


In [ ]:

out_path = Path("data/square_exponential_decomposition.json")
out_path.parent.mkdir(parents=True, exist_ok=True)

export = {
    "description": "Squared exponential covariance decomposition on [-5, 5]^2",
    "grid_extent": extent,
    "grid_shape": [grid_points, grid_points],
    "mean": mean_value,
    "variance": variance,
    "correlation_length": correlation_length,
    "grid_x": x.tolist(),
    "grid_y": y.tolist(),
    "eigenvalues": kept_evals.tolist(),
    # Store eigenvectors row-major per mode for compactness
    "eigenvectors": [vec.tolist() for vec in kept_evecs.T],
}

with out_path.open("w") as f:
    json.dump(export, f, indent=2)

out_path



## Loader class for evaluation and sampling
The class below reads the exported JSON, reconstructs the truncated basis, and exposes methods to evaluate fields from user-provided coefficients or to draw random samples.


In [ ]:

class SquareExponentialField:
    """Load and evaluate a truncated squared exponential decomposition."""

    def __init__(self, mean, eigenvalues, eigenvectors, grid_x, grid_y):
        self.mean = float(mean)
        self.eigenvalues = np.asarray(eigenvalues, dtype=float)
        self.sqrt_eigenvalues = np.sqrt(self.eigenvalues)
        # Eigenvectors stored as (modes, n_points)
        self.eigenvectors = np.asarray(eigenvectors, dtype=float)
        self.grid_x = np.asarray(grid_x, dtype=float)
        self.grid_y = np.asarray(grid_y, dtype=float)
        self.grid_shape = (len(self.grid_x), len(self.grid_y))

    @classmethod
    def from_json(cls, path):
        with Path(path).open() as f:
            data = json.load(f)
        eigenvectors = np.asarray(data["eigenvectors"], dtype=float)
        return cls(
            mean=data["mean"],
            eigenvalues=data["eigenvalues"],
            eigenvectors=eigenvectors,
            grid_x=data["grid_x"],
            grid_y=data["grid_y"],
        )

    def evaluate(self, coefficients=None):
        """Evaluate the truncated expansion on the stored grid.

        Args:
            coefficients: Optional array of shape (modes,) or (n_samples, modes).
                Defaults to zeros (producing the mean field).

        Returns:
            Field values with shape (grid_x.size, grid_y.size) if coefficients is 1D,
            or (n_samples, grid_x.size, grid_y.size) if 2D.
        """
        if coefficients is None:
            coefficients = np.zeros((self.eigenvectors.shape[0],), dtype=float)
        coeffs = np.asarray(coefficients, dtype=float)

        single_input = coeffs.ndim == 1
        if single_input:
            coeffs = coeffs[None, :]

        if coeffs.shape[1] != self.eigenvectors.shape[0]:
            raise ValueError("Coefficient dimension does not match number of modes")

        weighted_modes = coeffs * self.sqrt_eigenvalues[None, :]
        fields = weighted_modes @ self.eigenvectors
        fields = fields + self.mean

        if single_input:
            return fields.reshape(self.grid_shape)
        return fields.reshape((-1, *self.grid_shape))

    def sample(self, n_samples=1, rng=None):
        """Draw random field samples using standard normal coefficients."""
        rng = np.random.default_rng(rng)
        coeffs = rng.standard_normal(size=(n_samples, self.eigenvectors.shape[0]))
        return self.evaluate(coeffs)



## Example usage
Load the saved JSON, draw a couple of random samples, and inspect their shapes.


In [ ]:

field = SquareExponentialField.from_json("data/square_exponential_decomposition.json")

# Evaluate the mean field (all coefficients zero)
mean_field = field.evaluate()

# Draw a few random samples
few_samples = field.sample(n_samples=3, rng=0)

mean_field.shape, few_samples.shape
